# Build Your First Python Chatbot

Source: https://dzone.com/articles/python-chatbot-project-build-your-first-python-pro

## Introduction
Chatbots are extremely helpful for business organizations and also the customers. The majority of people prefer to talk directly from a chatbox instead of calling service centers. Facebook released data that proved the value of bots. More than 2 billion messages are sent between people and companies monthly. The HubSpot research tells us that 71% of people want to get customer support from messaging apps. It is a quick way to get their problems solved so chatbots have a bright future in organizations.

Today we are going to build an exciting project on Chatbot. We will implement a chatbot from scratch that will be able to understand what the user is talking about and give an appropriate response.

## Prerequisites
To implement the chatbot, we will be using Keras, which is a Deep Learning library, NLTK, which is a Natural Language Processing toolkit, and some helpful libraries. Run the below command to make sure all the libraries are installed:

In [1]:
#!pip install tensorflow keras pickle nltk

In [2]:
import pickle

## How do Chatbots Work?
Chatbots are nothing but an intelligent piece of software that can interact and communicate with people just like humans. Interesting, isn’t it? So now let's see how they actually work.

All chatbots come under the NLP (Natural Language Processing) concepts. NLP is composed of two things:

**NLU (Natural Language Understanding)**: The ability of machines to understand human language like English.

**NLG (Natural Language Generation)**: The ability of a machine to generate text similar to human written sentences.

Imagine a user asking a question to a chatbot: “Hey, what’s on the news today?”

The chatbot will break down the user sentence into two things: intent and an entity. The intent for this sentence could be get_news as it refers to an action the user wants to perform. The entity tells specific details about the intent, so "today" will be the entity. So this way, a machine learning model is used to recognize the intents and entities of the chat.



## Project File Structure
After the project is complete, you will be left with all these files. Lets quickly go through each of them. It will give you an idea of how the project will be implemented.

Train_chatbot.py — In this file, we will build and train the deep learning model that can classify and identify what the user is asking to the bot.

Gui_Chatbot.py — This file is where we will build a graphical user interface to chat with our trained chatbot.

Intents.json — The intents file has all the data that we will use to train the model. It contains a collection of tags with their corresponding patterns and responses.

Chatbot_model.h5 — This is a hierarchical data format file in which we have stored the weights and the architecture of our trained model.

Classes.pkl — The pickle file can be used to store all the tag names to classify when we are predicting the message.

Words.pkl — The words.pkl pickle file contains all the unique words that are the vocabulary of our model.

## How to Build Your Own Chatbot
I’ve simplified the building of this chatbot in 5 steps:

### Step 1. Import Libraries and Load the Data
Create a new python file and name it as train_chatbot and then we are going to import all the required modules. After that, we will read the JSON data file in our Python program.

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

nltk.download('punkt')
nltk.download('wordnet')

import json
import pickle

intents_file = open('intents.json').read()
intents = json.loads(intents_file)

2023-09-28 18:34:44.489727: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/reinaldolepsch/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/reinaldolepsch/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Step 2. Preprocessing the Data
The model cannot take the raw data. It has to go through a lot of pre-processing for the machine to easily understand. For textual data, there are many preprocessing techniques available. The first technique is tokenizing, in which we break the sentences into words.

By observing the intents file, we can see that each tag contains a list of patterns and responses. We tokenize each pattern and add the words in a list. Also, we create a list of classes and documents to add all the intents associated with patterns.

In [4]:
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
 
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)        

        #add documents in the corpus
        documents.append((word, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'), (['Give', 'm

Another technique is Lemmatization. We can convert words into the lemma form so that we can reduce all the canonical words. For example, the words play, playing, plays, played, etc. will all be replaced with play. This way, we can reduce the number of total words in our vocabulary. So now we lemmatize each word and remove the duplicate words.

In [5]:
# lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
87 unique lemmatized words ["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In the end, the words contain the vocabulary of our project and classes contain the total entities to classify. To save the python object in a file, we used the pickle.dump() method. These files will be helpful after the training is done and we predict the chats.

### Step 3. Create Training and Testing Data
To train the model, we will convert each input pattern into numbers. First, we will lemmatize each word of the pattern and create a list of zeroes of the same length as the total number of words. We will set value 1 to only those indexes that contain the word in the patterns. In the same way, we will create the output by setting 1 to the class input the pattern belongs to.

In [7]:
# create the training data
training = []

# create empty array for the output
output_empty = [0] * len(classes)

# training set, bag of words for every sentence
for doc in documents:

    # initializing bag of words
    bag = []

    # list of tokenized words for the pattern
    word_patterns = doc[0]

    # lemmatize each word - create base word, in attempt to represent related words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]

    # create the bag of words array with 1, if word is found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
        
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [11]:
training

[[[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0, 1, 0, 0, 0, 0, 0, 0, 0]],
 [[0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
 

In [15]:
!pip install numpy

In [16]:
import numpy as np

In [18]:
# shuffle the features and make numpy array
random.shuffle(training)
training = np.array(training, dtype=object)

In [19]:
# create training and testing lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data is created")

Training data is created


### Step 4. Training the Model
The architecture of our model will be a neural network consisting of 3 dense layers. The first layer has 128 neurons, the second one has 64 and the last layer will have the same neurons as the number of classes. The dropout layers are introduced to reduce overfitting of the model. We have used the SGD optimizer and fit the data to start the training of the model. After the training of 200 epochs is completed, we then save the trained model using the Keras model.save(“chatbot_model.h5”) function.

In [20]:
# deep neural networds model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compiling model. SGD with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#Training and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model is created")

/Users/reinaldolepsch/anaconda3/lib/python3.11/site-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
10/10 [==============================] - 0s 2ms/step - loss: 2.2218 - accuracy: 0.1277
Epoch 2/200
10/10 [==============================] - 0s 1ms/step - loss: 2.0916 - accuracy: 0.2766
Epoch 3/200
10/10 [==============================] - 0s 1ms/step - loss: 2.0046 - accuracy: 0.2766
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0037 - accuracy: 0.2766
Epoch 5/200
10/10 [==============================] - 0s 1ms/step - loss: 1.7875 - accuracy: 0.5106
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 1.8072 - accuracy: 0.4681
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.6118 - accuracy: 0.4255
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.4347 - accuracy: 0.7021
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.4931 - accuracy: 0.5745
Epoch 10/200
10/10 [==============================] - 0s 3ms/step - loss: 1.2627 - accuracy: 0.6596
Epoch 11/

10/10 [==============================] - 0s 1ms/step - loss: 0.0552 - accuracy: 0.9787
Epoch 84/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0388 - accuracy: 1.0000
Epoch 85/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0434 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0933 - accuracy: 0.9787
Epoch 87/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0927 - accuracy: 0.9787
Epoch 88/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0331 - accuracy: 1.0000
Epoch 89/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0277 - accuracy: 1.0000
Epoch 90/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0413 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0664 - accuracy: 0.9574
Epoch 92/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0837 - accuracy: 0.9787
Epoch 93/200


10/10 [==============================] - 0s 1ms/step - loss: 0.0084 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0889 - accuracy: 0.9574
Epoch 166/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0379 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0569 - accuracy: 0.9787
Epoch 168/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0397 - accuracy: 0.9787
Epoch 169/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0665 - accuracy: 0.9574
Epoch 170/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0399 - accuracy: 0.9787
Epoch 171/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0072 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0148 - accuracy: 1.0000
Epoch 173/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0326 - accuracy: 1.0000
Epoc

### Step 5. Interacting With the Chatbot
Our model is ready to chat, so now let’s create a nice graphical user interface for our chatbot in a new file. You can name the file as gui_chatbot.py

In our GUI file, we will be using the Tkinter module to build the structure of the desktop application and then we will capture the user message and again perform some preprocessing before we input the message into our trained model.

The model will then predict the tag of the user’s message, and we will randomly select the response from the list of responses in our intents file.

Here’s the full source code for the GUI file.

(ver o arquivo gui_chatbot.py já pronto)

## Running the Chatbot
Now we have two separate files, one is the train_chatbot.py, which we will use first to train the model.

python train_chatbot.py

In [22]:
!python train_chatbot.py

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'), (['Give', 'm

10/10 [==============================] - 0s 912us/step - loss: 0.1629 - accuracy: 0.9574
Epoch 45/200
10/10 [==============================] - 0s 994us/step - loss: 0.1922 - accuracy: 0.9574
Epoch 46/200
10/10 [==============================] - 0s 930us/step - loss: 0.0959 - accuracy: 1.0000
Epoch 47/200
10/10 [==============================] - 0s 969us/step - loss: 0.1677 - accuracy: 0.9787
Epoch 48/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1212 - accuracy: 0.9574
Epoch 49/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1211 - accuracy: 0.9787
Epoch 50/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1297 - accuracy: 0.9787
Epoch 51/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1033 - accuracy: 1.0000
Epoch 52/200
10/10 [==============================] - 0s 994us/step - loss: 0.2113 - accuracy: 0.9149
Epoch 53/200
10/10 [==============================] - 0s 987us/step - loss: 0.1232 - accuracy: 0.9574
E

10/10 [==============================] - 0s 986us/step - loss: 0.0066 - accuracy: 1.0000
Epoch 127/200
10/10 [==============================] - 0s 871us/step - loss: 0.0111 - accuracy: 1.0000
Epoch 128/200
10/10 [==============================] - 0s 958us/step - loss: 0.0429 - accuracy: 0.9787
Epoch 129/200
10/10 [==============================] - 0s 925us/step - loss: 0.0092 - accuracy: 1.0000
Epoch 130/200
10/10 [==============================] - 0s 873us/step - loss: 0.0201 - accuracy: 1.0000
Epoch 131/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0442 - accuracy: 0.9787
Epoch 132/200
10/10 [==============================] - 0s 980us/step - loss: 0.0189 - accuracy: 1.0000
Epoch 133/200
10/10 [==============================] - 0s 888us/step - loss: 0.0614 - accuracy: 0.9574
Epoch 134/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0386 - accuracy: 0.9787
Epoch 135/200
10/10 [==============================] - 0s 869us/step - loss: 0.0292 - accur

In [24]:
!python gui_chatbot.py